# GANによる異常検知

In [2]:
!gpustat

/bin/sh: gpustat: command not found


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [5]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
device = "cpu"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 6-3 Efficient GANの概要
AnoGANではテスト画像に対して異常検知をするのにまずテスト画像と最も近い画像を作ることのできる生成ノイズを学習する必要があり，異常検知に時間がかかってしまう問題があった．本節ではその問題を解決するためにEfficient GANを理解する

### Efficient GAN
Efficient GANではテスト画像から生成ノイズzを生成するエンコーダEを新たに作る．つまりこれは生成器Gとは逆の作用をしていて${\rm G}^{-1}$と表すことができる．このEについては通常通りGANを生成した後に独立に作ろうとすると上手くいかないらしい．なのでここではG,D,Eを同時に作る手順を理解する．

### Encoderを作る方法，後から作る作戦が上手くいかない理由
- [Adversarially learned inference](https://arxiv.org/pdf/1606.00704.pdf)
- [Adversarially learned inference GitHub page](https://ishmaelbelghazi.github.io/ALI/)


詳しくは上のサイトの図を参照するのがわかりやすい．それぞれの方法で作成したEncoderについて二次元データ(本来は画像を用いるところ)を入力して，生成ノイズzの空間に写像させた時に本来GANの入力ノイズは平均0,　分散1の均一な空間になると基づいて設計されているが適切な学習をさせていないEncoderの写像はそのような空間になっていない．つまりこれは平均0, 分散1の生成ノイズに対して上手く画像を生成できないことを意味している．

この直感的な理由の理解としては，Generatorが入力データの平均や分散を完全に理解していれば問題ないがそれは現実的には困難であり，不完全に学習したGeneratorを基にして学習するよりは本来学習の基にすべきである教師データを基にEncoderも学習するべきであるというように考えることができる．


### EncoderをGANと同時に作る方法
今回はEncoderを教師データの画像xと関与させるために**BiGAN(Bidiretional GAN)** と呼ばれる仕組みを利用する．
BiGANではDiscriminatorに対して$(x, E(x))$と$(G(z), z)$を入力する．

つまり元々のDCGANの損失関数とBiGANの損失関数を比較すると
- Discriminator
$$
-\sum_{i=1}^M\log D(x) -\sum_{j=1}^M \log(1-D(G(z))
$$
$$
-\sum_{i=1}^M[\log  D(x_i, E(x_i)-\sum_{j=1}^M  \log(1-D(G(z_j), z_j))]
$$

- Generator
$$
-\sum_{j=1}^M\log D(G(z_j))
$$
$$
-\sum_{j=1}^M\log D(G(z_j), z_j)
$$

ここでEncoderの損失関数を考えると，EncoderはDiscriminatorを騙せると嬉しいので，Discriminatorの損失関数にマイナスをかけてEncoderに関与するところだけを取り出すと
- Encoder

$\sum_{i=1}^M\log D(x_i, E(x_i))$
となる．しかしGeneratorと同様にこのままでは学習が進まないので以下のようにする．
$$
- \sum_{i=1}^M\log (1- D(x_i, E(x_i)))
$$
